## Emotion Analysis

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

Using TensorFlow backend.


In [2]:
from sklearn import metrics, cross_validation

/home/rishikesh/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Data imported from: https://inclass.kaggle.com/c/sa-emotions

In [3]:
data = pd.read_csv('../text_emotion.csv')
data.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


Only keeping the necessary columns.

In [4]:
# Keeping only the neccessary columns
data = data[['content','sentiment']]
data.head()

,content,sentiment
0,@tiffanylue i know i was listenin to bad habi...,empty
1,Layin n bed with a headache ughhhh...waitin o...,sadness
2,Funeral ceremony...gloomy friday...,sadness
3,wants to hang out with friends SOON!,enthusiasm
4,@dannycastillo We want to trade with someone w...,neutral


### Remove lessed labeled emotion for better prediction

In [5]:
del_row=['empty','enthusiasm','boredom','anger']

In [6]:
data=data[data['sentiment']!='empty']

In [7]:
data=data[data['sentiment']!='enthusiasm']

In [8]:
data=data[data['sentiment']!='boredom']

In [9]:
data=data[data['sentiment']!='anger']

In [10]:
data.shape

(38125, 2)

38125 data for 9 category prediction is very less encouraging 

In [11]:
MAX_NB_WORDS = 20000

In [12]:
MAX_SEQUENCE_LENGTH=30

In [13]:
data['content'] = data['content'].apply(lambda x: x.lower())
data['content'] = data['content'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data['sentiment'].value_counts())

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS, split=' ')
tokenizer.fit_on_texts(data['content'].values)
X = tokenizer.texts_to_sequences(data['content'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

neutral      8638
worry        8459
happiness    5209
sadness      5165
love         3842
surprise     2187
fun          1776
relief       1526
hate         1323
Name: sentiment, dtype: int64


/usr/local/lib/python3.5/dist-packages/keras/preprocessing/text.py:89: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [14]:
data['content']

1        layin n bed with a headache  ughhhhwaitin on y...
2                            funeral ceremonygloomy friday
4        dannycastillo we want to trade with someone wh...
5        repinging ghostridah14 why didnt you go to pro...
6        i should be sleep but im not thinking about an...
7                            hmmm httpwwwdjherocom is down
8                    charviray charlene my love i miss you
9                   kelcouch im sorry  at least its friday
10                                        cant fall asleep
11                                 choked on her retainers
12       ugh i have to beat this stupid song to get to ...
13       brodyjenner if u watch the hills in london u w...
14                                            got the news
15           the storm is here and the electricity is gone
16                                     annarosekerr agreed
17       so sleepy again and its not even that late i f...
18       perezhilton lady gaga tweeted about not being .

**The giving dataset contains lots of garbage words (*as obvious because people use slang and shortform at twitter due to 140 chars limitation*) as you see above, many word like layin,ughhhhwaitin,lovelylisaj,sendsome2me, n, niariley, mopedronin etc have no meaning which makes data noisy very noisy which ultimately give poor output**.

In [45]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Y = data['sentiment']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
Y=le.transform(Y) 
labels = to_categorical(np.asarray(Y))
print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', labels.shape)

VALIDATION_SPLIT=0.20

# split the data into a training set and a validation set
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X.shape[0])

x_train = X[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Found 51148 unique tokens.
Shape of data tensor: (38125, 2)
Shape of label tensor: (38125, 9)


In [46]:
embeddings_index = {}
f = open('glove.42B.300d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1917495 word vectors.


In [47]:
EMBEDDING_DIM=300

In [48]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [49]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [50]:
embed_dim = 300
lstm_out = 196

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.25))
model.add(Dense(9,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

C:\Anaconda3\lib\site-packages\keras\legacy\interfaces.py:86: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.25, recurrent_dropout=0.2)`
  '` call to the Keras 2 API: ' + signature)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 300)           15344700  
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               389648    
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 1773      
Total params: 15,736,121.0
Trainable params: 391,421.0
Non-trainable params: 15,344,700.0
_________________________________________________________________
None


Hereby I declare the train and test dataset.

In [79]:
# Y = pd.get_dummies(data['sentiment']).values
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
# print(X_train.shape,Y_train.shape)
# print(X_test.shape,Y_test.shape)

(25543, 32) (25543, 9)
(12582, 32) (12582, 9)


Here we train the Network. We should run much more than 7 epoch, but I would have to wait forever for kaggle, so it is 7 for now.

In [51]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Train on 30500 samples, validate on 7625 samples
Epoch 1/10
30500/30500 [==============================] - 50s - loss: 1.8402 - acc: 0.3323 - val_loss: 1.7368 - val_acc: 0.3656
Epoch 2/10
30500/30500 [==============================] - 49s - loss: 1.7260 - acc: 0.3745 - val_loss: 1.7034 - val_acc: 0.3794
Epoch 3/10
30500/30500 [==============================] - 49s - loss: 1.6785 - acc: 0.3912 - val_loss: 1.6928 - val_acc: 0.3819
Epoch 4/10
30500/30500 [==============================] - 49s - loss: 1.6503 - acc: 0.4017 - val_loss: 1.6828 - val_acc: 0.3870
Epoch 5/10
30500/30500 [==============================] - 49s - loss: 1.6207 - acc: 0.4116 - val_loss: 1.6899 - val_acc: 0.3847
Epoch 6/10
30500/30500 [==============================] - 49s - loss: 1.5922 - acc: 0.4197 - val_loss: 1.7012 - val_acc: 0.3761
Epoch 7/10
30500/30500 [==============================] - 49s - loss: 1.5626 - acc: 0.4304 - val_loss: 1.6891 - val_acc: 0.3878
Epoch 8/10
30500/30500 [==============================]

Extracting a validation set, and measuring score and accuracy.

In [82]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 2.38
acc: 0.33


Finally measuring the number of correct guesses.  It is clear that finding negative tweets goes very well for the Network but deciding whether is positive is not really. My educated guess here is that the positive training set is dramatically smaller than the negative, hence the "bad" results for positive tweets.

In [78]:
import xgboost as xgb
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(inp, target)
predictions = gbm.predict(X_test)
from sklearn import cross_validation
acc_scores = cross_validation.cross_val_score(gbm,inp, target, cv=5)
print (acc_scores)
print(metrics.accuracy_score(Y_test, predictions))

[ 0.27604849  0.27941658  0.27901639  0.28271564  0.28608924]
0.283016393443


In [76]:
from sklearn import linear_model
logisticReg = linear_model.LogisticRegression(multi_class='multinomial', solver='lbfgs')
logisticReg.fit(inp, target)
predictions = logisticReg.predict(X_test)
print(metrics.accuracy_score(Y_test, predictions))

0.258326776816


In [21]:
Y = pd.get_dummies(data['sentiment'])
print(Y.values)

[[0 0 1 ..., 0 0 0]
 [0 0 0 ..., 1 0 0]
 [0 0 0 ..., 1 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [77]:
Y = data['sentiment']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
Y=le.transform(Y) 
inp, X_test, target, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

In [69]:
Y

array([6, 6, 4, ..., 3, 1, 3])